In [ ]:
import time
import cfbd
from cfbd.rest import ApiException
import sportsdataverse
from pprint import pprint
import pandas as pd
import re

# Defining the host is optional and defaults to https://api.collegefootballdata.com
# See configuration.py for a list of all supported configuration parameters.
configuration = cfbd.Configuration(
    host = "https://api.collegefootballdata.com"
)

# The client must configure the authentication and authorization parameters
# in accordance with the API server security policy.
# Examples for each auth method are provided below, use the example that
# satisfies your auth use case.

# Configure Bearer authorization: apiKey
configuration = cfbd.Configuration(
    access_token = "6mc3nfaaZeCmf4Uh+G6AkVrmQoUbSSMbGUOE7OYcengOfBnHs7Zx08hFQcc1fbXU"
)

# Getting Draft Picks

In [ ]:
draft_pick_data_dicts = []

with cfbd.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = cfbd.DraftApi(api_client)
    years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
    position = 'WR'

    for year in years:
      try:
          api_response = api_instance.get_draft_picks(year=year, position=position)
          for response in api_response[:10]:
            draft_pick_data_dicts.append({"college_id": response.college_athlete_id, "name": response.name, "height": response.height, "weight": response.weight})
      except Exception as e:
          print("Exception when calling DraftApi->get_draft_picks: %s\n" % e)

draft_pick_df = pd.DataFrame(draft_pick_data_dicts)

In [ ]:
draft_pick_df

,college_id,name,height,weight
0,534458,Amari Cooper,73,211
1,546052,Kevin White,75,215
2,511615,DeVante Parker,75,209
3,531010,Nelson Agholor,72,198
4,531670,Breshad Perriman,74,212
...,...,...,...,...
95,4428209,Ricky Pearsall,73,189
96,4430034,Xavier Legette,73,221
97,4635008,Keon Coleman,75,213
98,4612826,Ladd McConkey,72,186


In [ ]:
draft_pick_df.to_csv('wr_draft_picks_2015_2024.csv', index=False)

# Getting PBP (2012-2021)

In [ ]:
pbp_df_list = []

columns = [
    "id", "sequenceNumber", "text", "awayScore", "homeScore",
    "statYardage", "type.id", "type.abbreviation", "period.number",
    "start.down", "start.distance", "start.yardLine", "sp",
    "start.yardsToEndzone", "start.possessionText", "start.team.id",
    "game_id", "season", "seasonType", "homeTeamId", "awayTeamId",
    "homeTeamAbbrev", "awayTeamAbbrev", "homeTeamSpread", "clock.minutes",
    "clock.seconds", "playType", "week", "pos_team", "pass_attempt",
    "completion", "target", "pass_breakup", "scoring_opp", "rz_play",
    "passing_down", "EPA_pass", "EPA_explosive_pass", "first_down_created",
    "EPA_success_pass", "wpa", "havoc", "pass_breakup"
]

for year in range(2012, 2022):
  pbp_df_list.append(sportsdataverse.cfb.load_cfb_pbp(seasons=year, return_as_pandas=True)[columns])

pbp_df = pd.concat(pbp_df_list)

100%|██████████| 1/1 [00:01<00:00,  1.33s/it]


In [ ]:
pattern = (
    r"pass (?:in)?complete to "
    r"(?P<receiver>null|[A-Za-z'’]+(?:-[A-Za-z'’]+)? [A-Za-z'’]+(?:-[A-Za-z'’]+)?)"
)

pbp_df["targeted_receiver"] = (
    pbp_df["text"]
        .str.extract(pattern)
        .where(pbp_df["target"], "")
)

In [ ]:
pbp_passing = pbp_df[pbp_df["target"] == True]

In [ ]:
pbp_passing.to_csv("college_pass_pbp_2012_2021.csv", index=False)